In [1]:
import pandas as pd
import yfinance as yf

In [2]:
nba = pd.ExcelFile('./raw_data/nba.xlsx')

In [3]:
nba.sheet_names

['Team Revenue - Total',
 'Team Revenue - Ticket',
 'Operating Income',
 'NBA Revenue',
 'Metros',
 'Season Data',
 'Dates',
 'Data Sources']

In [4]:
total = nba.parse('Team Revenue - Total')
total = total.rename({'Unnamed: 0': 'Team'}, axis=1)
total = total.set_index('Team')
total = total.unstack().reset_index()  
total = total.rename({'level_0': 'Year', 0: 'Total Revenue'}, axis=1)

In [5]:
ticket = nba.parse('Team Revenue - Ticket')
ticket = ticket.rename({'Unnamed: 0': 'Team'}, axis=1)
ticket = ticket.set_index('Team')
ticket = ticket.unstack().reset_index()  
ticket = ticket.rename({'level_0': 'Year', 0: 'Ticket Revenue'}, axis=1)

In [6]:
oi = nba.parse('Operating Income')
oi = oi.rename({'Unnamed: 0': 'Team'}, axis=1)
oi = oi.set_index('Team')
oi = oi.unstack().reset_index()  
oi = oi.rename({'level_0': 'Year', 0: 'Operating Income'}, axis=1)

In [7]:
total = pd.merge(total, ticket)
total = pd.merge(total, oi)

In [8]:
total

,Year,Team,Total Revenue,Ticket Revenue,Operating Income
0,2011,Atlanta Hawks,109,26.30,-15
1,2011,Boston Celtics,146,52.70,8
2,2011,Brooklyn Nets,89,23.30,-24
3,2011,Charlotte Hornets,101,19.50,-26
4,2011,Chicago Bulls,185,59.00,59
...,...,...,...,...,...
325,2021,Sacramento Kings,192,0.00,25
326,2021,San Antonio Spurs,205,2.37,39
327,2021,Toronto Raptors,194,1.62,2
328,2021,Utah Jazz,262,5.33,96


In [9]:
pops = nba.parse('Metros')
pops = pops[['Team',
             'Population, 2020',
             'Population, 2010',
             'Per Capita Income, 2010',
             'Per Capita Income, 2020']]

In [10]:
total = pd.merge(total, pops)
total['Population'] = None
total['Income'] = None

In [11]:
total.loc[total['Year'] <= 2020, 'Population'] = total.loc[total['Year'] <= 2020, 'Population, 2010']
total.loc[total['Year'] > 2020, 'Population'] = total.loc[total['Year'] > 2020, 'Population, 2020']

total.loc[total['Year'] <= 2020, 'Income'] = total.loc[total['Year'] <= 2020, 'Per Capita Income, 2010']
total.loc[total['Year'] > 2020, 'Income'] = total.loc[total['Year'] > 2020, 'Per Capita Income, 2020']

In [12]:
total = total.drop(['Population, 2020',
                    'Population, 2010',
                    'Per Capita Income, 2010',
                    'Per Capita Income, 2020'], axis=1)

In [13]:
seasons = nba.parse('Season Data')
seasons

,Team,Age,W,L,Attend.,Attend./G,Playoffs,Conference Semifinals,Conference Finals,Finals,Championship,Year
0,Orlando Magic,27.9,59,23,715901,17461,1.0,1.0,1,0,0.0,2010
1,Cleveland Cavaliers,28.0,61,21,843042,20562,1.0,1.0,0,0,0.0,2010
2,Utah Jazz,25.6,53,29,794512,19378,1.0,1.0,0,0,0.0,2010
3,San Antonio Spurs,28.7,50,32,741676,18090,1.0,1.0,0,0,0.0,2010
4,Atlanta Hawks,26.5,53,29,678375,16546,1.0,1.0,0,0,0.0,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
385,Detroit Pistons,23.6,23,59,663556,16184,0.0,0.0,0,0,0.0,2022
386,Orlando Magic,23.3,22,60,622881,15192,0.0,0.0,0,0,0.0,2022
387,Oklahoma City Thunder,22.4,24,58,595112,14515,0.0,0.0,0,0,0.0,2022
388,Houston Rockets,24.1,20,62,638977,15585,0.0,0.0,0,0,0.0,2022


In [14]:
data = pd.merge(total, seasons, how='left')

In [15]:
data['nGames'] = data['W'] + data['L']

In [16]:
data['WP'] = data['W'] / data['nGames']

In [17]:
data = data.drop(['W', 'L'], axis=1)

In [18]:
data = data.rename({'Total Revenue': 'Revenue',
                    'Ticket Revenue': 'Ticket',
                    'Operating Income': 'OI',
                    'Attend.': 'Attendance',
                    'Attend./G': 'APG',
                    'Conference Semifinals': 'CSF',
                    'Conference Finals': 'CF'}, axis=1)

In [19]:
data['Revenue'] = data['Revenue'] * 82 / data['nGames']
data['Ticket'] = data['Ticket'] * 82 / data['nGames']

In [20]:
data = data.drop(['Attendance', 'nGames'], axis=1)

In [21]:
dates = nba.parse('Dates')

In [22]:
pend2start = []
pstart2start = []
newyear2start = []

In [23]:
for i in range(dates.shape[0]):
    start = dates.loc[i, 'Previous End']
    end = dates.loc[i, 'Start']
    temp_data = yf.download('SPY', start = start.date(), end = end.date())
    pend2start.append((temp_data.values[-1,3] - temp_data.values[0,0]) / temp_data.values[0,0])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [24]:
for i in range(dates.shape[0]):
    start = dates.loc[i, 'Previous Start']
    end = dates.loc[i, 'Start']
    temp_data = yf.download('SPY', start = start.date(), end = end.date())
    pstart2start.append((temp_data.values[-1,3] - temp_data.values[0,0]) / temp_data.values[0,0])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [25]:
for i in range(dates.shape[0]):
    start = dates.loc[i, 'BoY']
    end = dates.loc[i, 'Start']
    temp_data = yf.download('SPY', start = start.date(), end = end.date())
    newyear2start.append((temp_data.values[-1,3] - temp_data.values[0,0]) / temp_data.values[0,0])

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [26]:
dates['MPC1'] = pend2start
dates['MPC2'] = pstart2start
dates['MPC3'] = newyear2start

In [27]:
data = pd.merge(data,
                dates[['Season', 'MPC1', 'MPC2', 'MPC3']],
                how='left',
                left_on='Year',
                right_on='Season') 

In [28]:
data = data.drop('Season', axis=1)

In [29]:
social = pd.read_csv('./Raw Data/social.csv')

In [30]:
data = pd.merge(data, social[['Team', 'Year', 'Followers', 'Franch_Val']], how='left', on=['Team', 'Year']) 

In [31]:
data

,Year,Team,Revenue,Ticket,OI,Population,Income,Age,APG,Playoffs,CSF,CF,Finals,Championship,WP,MPC1,MPC2,MPC3,Followers,Franch_Val
0,2011,Atlanta Hawks,109.000000,26.300000,-15,5286728,38774,27.4,15649,1.0,1.0,0,0,0.0,0.536585,0.057178,0.109035,0.056332,0.20,295
1,2012,Atlanta Hawks,123.000000,24.972727,-19,5286728,38774,27.9,15200,1.0,0.0,0,0,0.0,0.606061,-0.011729,0.070195,-0.002525,0.35,270
2,2013,Atlanta Hawks,119.000000,23.600000,-4,5286728,38774,27.1,15126,1.0,0.0,0,0,0.0,0.536585,0.073273,0.120314,0.106371,0.52,316
3,2014,Atlanta Hawks,133.000000,22.900000,15,5286728,38774,27.5,14231,1.0,0.0,0,0,0.0,0.463415,0.127583,0.242369,0.214458,1.20,425
4,2015,Atlanta Hawks,142.000000,27.800000,7,5286728,38774,27.8,17412,1.0,1.0,1,0,0.0,0.731707,0.057637,0.110570,0.066203,1.78,825
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
325,2017,Washington Wizards,222.000000,55.100000,21,5636232,58429,26.0,17003,1.0,1.0,0,0,0.0,0.597561,0.029068,0.042144,0.071824,2.29,"1,000"
326,2018,Washington Wizards,255.000000,56.800000,34,5636232,58429,26.9,18032,1.0,0.0,0,0,0.0,0.524390,0.048032,0.189165,0.134421,2.38,"1,350"
327,2019,Washington Wizards,269.000000,60.900000,55,5636232,58429,26.5,17448,0.0,0.0,0,0,0.0,0.390244,-0.003957,0.075109,0.024492,2.55,"1,550"
328,2020,Washington Wizards,263.083333,48.516667,39,5636232,58429,25.1,16647,0.0,0.0,0,0,0.0,0.347222,0.042464,0.084562,0.219571,5.74,"1,750"


In [32]:
data.to_csv('cleaned.csv')